In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import cv2
import json

# Define VGG-16 Net

In [2]:
from kaffe.tensorflow import Network

class VGG_ILSVRC_16_layers(Network):
    def setup(self):
        (self.feed('input')
             .conv(3, 3, 64, 1, 1, name='conv1_1')
             .conv(3, 3, 64, 1, 1, name='conv1_2')
             .max_pool(2, 2, 2, 2, name='pool1')
             .conv(3, 3, 128, 1, 1, name='conv2_1')
             .conv(3, 3, 128, 1, 1, name='conv2_2')
             .max_pool(2, 2, 2, 2, name='pool2')
             .conv(3, 3, 256, 1, 1, name='conv3_1')
             .conv(3, 3, 256, 1, 1, name='conv3_2')
             .conv(3, 3, 256, 1, 1, name='conv3_3')
             .max_pool(2, 2, 2, 2, name='pool3')
             .conv(3, 3, 512, 1, 1, name='conv4_1')
             .conv(3, 3, 512, 1, 1, name='conv4_2')
             .conv(3, 3, 512, 1, 1, name='conv4_3')
             .max_pool(2, 2, 2, 2, name='pool4')
             .conv(3, 3, 512, 1, 1, name='conv5_1')
             .conv(3, 3, 512, 1, 1, name='conv5_2')
             .conv(3, 3, 512, 1, 1, name='conv5_3')
             .max_pool(2, 2, 2, 2, name='pool5')
             .fc(4096, name='fc6')
             .fc(4096, name='fc7')
             .fc(1000, relu=False, name='fc8')
             .softmax(name='prob'))

In [3]:
inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
# initialize VGG-16 net
net = VGG_ILSVRC_16_layers({'input': inputs})
# we only need to extract fc7 layer features
fc7 = net.layers['fc7']

In [4]:
def load_images(image_list):
    """Load images into numpy arrays from an image_list."""
    # specific for vgg model, subtract mean pixel value
    mean_pixel = [103.939, 116.779, 123.68]
    image_full_batch = []
    for image_path in image_list:
        # 1 means read color image
        image_tensor = cv2.imread(image_path, 1).astype('float32')
        image_full_batch.append(image_tensor)
    # put all image tensors together into the full batch
    image_full_batch = np.stack(image_full_batch)
    for c in xrange(3):
        image_full_batch[:, :, :, c] -= mean_pixel[c]
    return image_full_batch

In [5]:
def generate_batch(inputs, batch_size=100):
    """Generate mini batches for given inputs."""
    full_size = inputs.shape[0]
    iterations = full_size // batch_size
    count = 0
    while count < iterations:
        yield inputs[count*batch_size:(count+1)*batch_size]
        count +=1
    # finally, yield data that has not been used yet 
    # maybe we should check the size, in case that there is only one example left (dims decreases)
    if count*batch_size != full_size:
        yield inputs[count*batch_size:]

In [6]:
def extract_image_features(images):
    """Extract fc7 features using vgg-16 model."""
    with tf.Session() as sess:
        # Load the data
        sess.run(tf.initialize_all_variables())
        net.load('vgg_16.tfmodel', sess)

        # train_image_embeddings
        image_features = []
        for image_batch in generate_batch(images, batch_size = 200):
            feed = {inputs: image_batch}
            batch_image_features = sess.run([fc7], feed_dict=feed)
            # notice the 0, since the dimension of bacth_image_features is 1xbatch_sizex4096
            image_features.append(batch_image_features[0])
            
    image_features = np.vstack(image_features)
    return image_features

# Load images

In [7]:
with open('Corel5k/train_image_list.json') as f:
    train_image_list = json.load(f)
train_images = load_images(train_image_list)

with open('Corel5k/test_image_list.json') as f:
    test_image_list = json.load(f)
test_images = load_images(test_image_list)

# Extract image features and save for later use

In [8]:
train_image_features = extract_image_features(train_images)
with open("train_image_features.npy", "w") as f:
    np.save(f, train_image_features)

test_image_features = extract_image_features(test_images)
with open("test_image_features.npy", "w") as f:
    np.save(f, test_image_features)

In [ ]:
train_image_features.shape